# Example test code
The following code showcases how exactly to use the simulator

In [1]:
# autoreloading magic
%load_ext autoreload
%autoreload 2

# this adds the parent directory into the path, since we want simulations from the parent directory
import sys; sys.path.insert(0, '..') 

import pandas as pd
import numpy as np
from simulator import Simulator, Demand, run_simulations
from resources import BasicResource, ResourcePackage, BasicCallData, BasicGas, Basefee
from resources import IndependentResources, CorrelatedResources, IndividualResources, JointResources
from oracle import Oracle
import matplotlib.pyplot as plt

# save into hdf5 file
import os
import h5py
import uuid

# Instructions
1. Create your basefee object. If your resource is of type split, create only one basefee object and ResourcePackage will automatically split it for you. Else, follow instructions below
2. Create your ResourcePackage. Options include: IndependentResources, CorrelatedResources, IndividualResources, and JointResources
3. Create your Demand class and pass in ResourcePackage as one of the variables
4. Create your Simulator class with only the Demand class as the input. All relevant data will have been extracted from previous inputs.
5. Run sim.simulation(step_count) to evolve the system step_count number of times

## Independent Resources (X+Y =Z)
Here, we generate one pareto value and split it randomly such that both resources are uncorrelated. The ratio parameter doesn't really matter, it exists for backwards compatibility

In [2]:
bf_standard_value = 38.100002694
bf_standard = Basefee(1.0 / 8, 15000000, 30000000, bf_standard_value)
resource_package = IndependentResources(["gas","call_data"],[0.3,0.7],bf_standard)
demand = Demand(2000, 0, 400, resource_package)
sim = Simulator(demand)
sim.simulate(10)

({'gas': [38.100002694,
   41.92228432146961,
   47.15801643394049,
   53.052726661420245,
   50.03838895611565,
   55.38706311439241,
   53.81489457525159,
   53.252984322189874,
   53.76003493236549,
   51.310659200429285,
   56.66908982912148],
  'call_data': [38.100002694,
   42.861061560284206,
   44.66694154217274,
   48.97349875410343,
   46.03185573552304,
   51.78477433285303,
   49.308743939242014,
   48.0420920875449,
   48.44648012603403,
   45.71323096633874,
   51.42311395160921]},
 {'blocks': [[{'gas price': 54.91082124232343,
     'gas limit': 546928.3028941753,
     'call_data price': 54.91082124232343,
     'call_data limit': 4777650.7525624735,
     'time': 0.0,
     'total_value': 292377008.70479935,
     'profit': 89510532.34748507},
    {'gas price': 57.012229988585545,
     'gas limit': 12301.975360944765,
     'call_data price': 57.012229988585545,
     'call_data limit': 1369231.705669819,
     'time': 0.0,
     'total_value': 78764315.95990308,
     'profit': 

## Correlated Resources (X+Y=Z)
Here, we generate one pareto value and split it with the given ratio. Eg, in this case 30% is gas and 70% is call data

In [3]:
bf_standard_value = 38.100002694
bf_standard = Basefee(1.0 / 8, 15000000, 30000000, bf_standard_value)
resource_package = CorrelatedResources(["gas","call_data"],[0.3,0.7],bf_standard)
demand = Demand(2000, 0, 400, resource_package)
sim = Simulator(demand)
sim.simulate(10)

({'gas': [38.100002694,
   42.86224288922869,
   48.213617544753305,
   50.206016120341765,
   47.291277754381554,
   51.24218628843658,
   45.49749120228168,
   49.873238200106314,
   53.028185152493265,
   46.542656925749746,
   46.76213673387413],
  'call_data': [38.100002694,
   42.86224288922868,
   48.213617544753276,
   50.20601612034173,
   47.29127775438152,
   51.242186288436535,
   45.497491202281644,
   49.873238200106286,
   53.02818515249324,
   46.542656925749725,
   46.76213673387411]},
 {'blocks': [[{'gas price': 58.57297071744128,
     'gas limit': 279863.8495863857,
     'call_data price': 58.57297071744128,
     'call_data limit': 653015.6490348999,
     'time': 0.0,
     'total_value': 54641523.55564586,
     'profit': 19098812.144997515},
    {'gas price': 62.053773366436765,
     'gas limit': 206250.87693147644,
     'call_data price': 62.053773366436765,
     'call_data limit': 481252.046173445,
     'time': 0.0,
     'total_value': 42662150.5791156,
     'profi

# Individual Resources (X+Y)
Here, we generate 2 resources individually with no given ratio. They can have an individual basefee for each resource and work without a basefee max limit.

In [4]:
bf_standard_value = 38.100002694
bf_standard = Basefee(1.0 / 8, 15000000, 30000000, bf_standard_value)
resource_package = IndividualResources([BasicGas(bf_standard),BasicCallData(bf_standard)])
demand = Demand(2000, 0, 400, resource_package)
sim = Simulator(demand)
sim.simulate(10)

({'gas': [38.100002694,
   42.790435641926145,
   42.10871119325865,
   41.46186001898509,
   40.77220988823706,
   40.12793783815372,
   39.51345517773235,
   38.907720524564446,
   38.297501167075566,
   37.708674060301185,
   37.0834960072813],
  'call_data': [38.100002694,
   37.44587963663722,
   36.859751653828155,
   36.28669879741006,
   35.67976716225641,
   35.131484066221425,
   34.58652295470558,
   34.05555965944677,
   33.518868575056494,
   33.00367527371419,
   32.462793812023776]},
 {'blocks': [[{'gas price': 65.4655048746086,
     'gas limit': 1355791.9341903906,
     'call_data price': 65.4655048746086,
     'call_data limit': 4.75977390515793,
     'time': 0.0,
     'total_value': 88757915.07769783,
     'profit': 37102057.38514187},
    {'gas price': 56.825832533997605,
     'gas limit': 1859230.4886013027,
     'call_data price': 56.825832533997605,
     'call_data limit': 240.00785659477856,
     'time': 0.0,
     'total_value': 105665959.03362587,
     'profit':

## Joint Resources
This is building upon Adam's code of generating data points jointly. More operational details should be referred to resources.py

In [7]:
# Basefee initialization for gas
bf_standard_value = 38.100002694
bf_standard = Basefee(1.0 / 8, 15000000, 30000000, bf_standard_value)

# Basefee initialization for calldata
call_data_standard_value = 25
call_data_learning_rate = 1.0 / 8
call_data_target_limit = 25000
call_data_max_limit = call_data_target_limit * 2
call_data_standard = Basefee(call_data_learning_rate, call_data_target_limit, call_data_max_limit, call_data_standard_value)

special_generation_file = "specialGenerationNFT.csv"

j_r = JointResources(["gas", "call_data"], bf_standard, call_data_standard,filename=special_generation_file)
demand = Demand(2000, 0, 400, j_r)
sim = Simulator(demand,knapsack_solver="greedy",tx_decay_time=-1)

step_count = 10

basefees_data, block_data, mempools_data, basefees_stats = sim.simulate(step_count)
data = {
    "basefees_data": basefees_data,
    "block_data": block_data,
    "mempools_data": mempools_data,
    "basefees_stats": basefees_stats,
}

## Generating multiple simulations and taking its average
Here is example code showing how to run these simulations over multiple iterations. Simply pass in the Simulator object into run_simulations and it will output individual and averaged data files and plot the final average basefee over time.

In [ ]:
j = JointResources(["gas","call_data"])
demand = Demand(2000, 0, 400, j)
sim = Simulator(demand)
run_simulations(sim,10,2,"hdf5+csv") # Simulator, step_count, num_iterations, filetype, filepath


Some unused model fitting code

In [ ]:
# Model fitting
from lmfit import Model, Parameters, minimize, report_fit
def Exponential(x, A, sigma, c=None):
    if c == None:
        c = 0
    return A * np.exp(-(x / sigma)) + c

x_array = [x for x in range(len(gas_average))]

gmodel = Model(Exponential)
params = Parameters()
params.add('A', value=40, vary=True)
params.add('alpha', value=2, vary=True)
params.add('sigma', value=100, vary=True)
gresult = gmodel.fit(gas_average, params, x=x_array)

# Plot
plt.plot(x_array, gresult.best_fit, '-', label="Best Fit")
plt.plot(x_array, gas_average, label="Data")
plt.legend()
plt.show()

print(gresult.result)